In [52]:
import datetime
import requests
import pandas as pd

In [53]:
codigoINE_renta = {
    "Bizkaia" : 30918,
    "Gipuzkoa" : 31008
}

hours_added = datetime.timedelta(hours = 1)
num_datos = 50000
url_plantilla = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{codigo}?nult={num_datos}'

In [54]:
df_renta = pd.DataFrame(columns=[])

for provincia in codigoINE_renta:
    main_url_INE_Estructura_Datos = "https://servicios.ine.es/wstempus/js/ES/SERIES_TABLA/" + str(codigoINE_renta[provincia]) + "?page=1"
    INE_Estructura_Datos = requests.get(main_url_INE_Estructura_Datos).json()

    df_INE_Estructura_Datos_Renta = pd.DataFrame.from_dict(INE_Estructura_Datos)
    
    new = df_INE_Estructura_Datos_Renta["Nombre"].str.split('.')
    df_INE_Estructura_Datos_Renta["Nombre"] = new.map(lambda x: x[0])
    df_INE_Estructura_Datos_Renta["Tipo Renta"] = new.map(lambda x: x[2])
    
    df_INE_Estructura_Datos_Total_Renta = df_INE_Estructura_Datos_Renta[~df_INE_Estructura_Datos_Renta["Nombre"].str.contains("sección|distrito")]
    df_INE_Estructura_Datos_Seccion_Renta = df_INE_Estructura_Datos_Renta[df_INE_Estructura_Datos_Renta["Nombre"].str.contains("sección")]
    df_INE_Estructura_Datos_Distrito_Renta = df_INE_Estructura_Datos_Renta[df_INE_Estructura_Datos_Renta["Nombre"].str.contains("distrito")]
    
    df_INE_Estructura_Datos_Total_Renta["Tipo"] = "Total"
    df_INE_Estructura_Datos_Total_Renta = df_INE_Estructura_Datos_Total_Renta[["Id","COD","Nombre","Tipo","Tipo Renta"]]
    #display(df_INE_Estructura_Datos_Total_Renta)
    
    new2 = df_INE_Estructura_Datos_Seccion_Renta["Nombre"].str.split('sección')
    df_INE_Estructura_Datos_Seccion_Renta["Tipo"] = new2.map(lambda x: "sección " + x[1].strip())
    df_INE_Estructura_Datos_Seccion_Renta["Nombre"] = new2.map(lambda x: x[0].strip())
    df_INE_Estructura_Datos_Seccion_Renta = df_INE_Estructura_Datos_Seccion_Renta[["Id","COD","Nombre","Tipo","Tipo Renta"]]
    #display(df_INE_Estructura_Datos_Seccion_Renta)
    
    new3 = df_INE_Estructura_Datos_Distrito_Renta["Nombre"].str.split('distrito')
    df_INE_Estructura_Datos_Distrito_Renta["Tipo"] = new3.map(lambda x: "distrito " + x[1].strip())
    df_INE_Estructura_Datos_Distrito_Renta["Nombre"] = new3.map(lambda x: x[0].strip())
    df_INE_Estructura_Datos_Distrito_Renta = df_INE_Estructura_Datos_Distrito_Renta[["Id","COD","Nombre","Tipo","Tipo Renta"]]
    #display(df_INE_Estructura_Datos_Distrito_Renta)
    
    df_INE_Estructura_Datos_Renta_New = pd.concat([df_INE_Estructura_Datos_Total_Renta, df_INE_Estructura_Datos_Seccion_Renta, df_INE_Estructura_Datos_Distrito_Renta], axis=0)
    df_INE_Estructura_Datos_Renta_New = df_INE_Estructura_Datos_Renta_New.sort_values(by=['Nombre', "Tipo"])
    #display(df_INE_Estructura_Datos_Renta_New)
    #df_INE_Estructura_Datos_Renta_New.to_excel("df_INE_Estructura_Datos_Renta_New.xlsx")
    
    for index, poblacion in df_INE_Estructura_Datos_Renta_New.iterrows():
        codigo = poblacion["COD"]
        url_datos = url_plantilla.format(codigo=codigo, num_datos=num_datos)
        df_renta_ = pd.DataFrame.from_dict(requests.get(url_datos).json()["Data"])
        
        if len(list(df_renta_.columns)) > 0:
            df_renta_ = df_renta_[["Anyo","Fecha","Valor"]].rename(columns={"Anyo": "Año", "Fecha": "Fecha", "Valor": poblacion["Tipo Renta"].strip()})
            df_renta_["Fecha"] = pd.to_datetime(df_renta_["Fecha"], unit='ms')
            df_renta_["Fecha"] = df_renta_["Fecha"].map(lambda x: x + hours_added)
            df_renta_['Mes'] = df_renta_['Fecha'].dt.month
            df_renta_ = df_renta_[["Año", "Mes", poblacion["Tipo Renta"].strip()]]
            df_renta_ = df_renta_.set_index(["Año", "Mes"])
            #display(df_renta_)
            
            df_renta_.columns = pd.MultiIndex.from_product([ [provincia], [poblacion["Nombre"].strip()], [poblacion["Tipo"].strip()], df_renta_.columns], names=["Provincia", "Población","Tipo", "Tipo Renta"])
            
            df_renta = pd.concat([df_renta, df_renta_], axis=1)


C:\Users\abediaga\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\abediaga\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\abediaga\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [55]:
display(df_renta)

Provincia                     Bizkaia                                        \
Población                     Abadiño                                         
Tipo                            Total                                         
Tipo Renta Fuente de ingreso: salario Fuente de ingreso: otras prestaciones   
Año  Mes                                                                      
2015 1                        10847.8                                1089.1   
2016 1                        11401.4                                1093.4   
2017 1                        11549.9                                1115.6   

Provincia                                                                \
Población                                                                 
Tipo                                                        distrito 01   
Tipo Renta Fuente de ingreso: otros ingresos Fuente de ingreso: salario   
Año  Mes                                                                  
2015 1                                1626.7                    10847.8   
2016 1                                1560.2                    11401.4   
2017 1                                1490.9                    11549.9   

Provincia                                         \
Población                                          
Tipo                                               
Tipo Renta Fuente de ingreso: otras prestaciones   
Año  Mes                                           
2015 1                                    1089.1   
2016 1                                    1093.4   
2017 1                                    1115.6   

Provincia                                                                \
Población                                                                 
Tipo                                                      sección 01001   
Tipo Renta Fuente de ingreso: otros ingresos Fuente de ingreso: salario   
Año  Mes                                                                  
2015 1                                1626.7                    12974.7   
2016 1                                1560.2                    13945.0   
2017 1                                1490.9                    13873.4   

Provincia                                         \
Población                                          
Tipo                                               
Tipo Renta Fuente de ingreso: otras prestaciones   
Año  Mes                                           
2015 1                                    1089.1   
2016 1                                    1093.4   
2017 1                                    1115.6   

Provincia                                                                ...  \
Población                                                                ...   
Tipo                                                      sección 01002  ...   
Tipo Renta Fuente de ingreso: otros ingresos Fuente de ingreso: salario  ...   
Año  Mes                                                                 ...   
2015 1                                2721.8                     8130.2  ...   
2016 1                                2683.6                     8658.0  ...   
2017 1                                2300.9                     8809.2  ...   

Provincia                                                                \
Población                           Zierbena            Ziortza-Bolibar   
Tipo                           sección 01001                      Total   
Tipo Renta Fuente de ingreso: otros ingresos Fuente de ingreso: salario   
Año  Mes                                                                  
2015 1                                2221.8                    10339.0   
2016 1                                2034.5                    10770.3   
2017 1                                2553.1                    10919.2   

Provincia                                         \
Población                            

In [56]:
df_renta.to_excel("output_final.xlsx")